# Basic implementation

- load file
  - ~~load file with encoded function~~
  - ~~turn it into a numpy array~~
- noise (preprocessing)
  - scale vector into 0-1 range
  - add noise into it vectors values
  - (test) rescale vector into values range 0-{whatever my dictionary size is}
- pass it through a model
  - create a simple model (does not have to be diffusion)
  - (optional) develop loss function
  - (optional) implement diffusion model

### <span style="color:red"> IMPORTANT </span>
- parsing files results in more files than the original (file per function in file). should consider it while loading dataset
  - ~~split *.c files into functions~~
  - clean *.c files from unnecessary white spaces (most likely the same way it is done in lexer)

## Loading vectors

In [14]:
import numpy as np
import os
from multiprocessing.pool import ThreadPool

def prepare_record(c_file, parsed_file):
  parsed = np.loadtxt(parsed_file, dtype=float)
  return c_file, parsed

'''
  Because code files weight very little they can all be loaded at once
'''
c_dir = "./data/JL/"
parsed_dir = "./data/parsed/" # might change ofc

'''
  returns: array of pairs (c_file, encoded_c_file)
'''
def load_dataset(c_dir, parsed_dir) -> list:
  c_files = sorted(os.listdir(c_dir))
  parsed_files = sorted(os.listdir(parsed_dir))

  with ThreadPool() as pool:
    c_files = map(lambda f: f"{c_dir}{f}", c_files)
    parsed_files = map(lambda f: f"{parsed_dir}{f}", parsed_files)

  file_paths = zip(c_files, parsed_files)

  with ThreadPool() as pool:
    return [pool.map(lambda pair: prepare_record(*file_paths), file_paths)]

In [ ]:
dataset = load_dataset(c_dir, parsed_dir)
dataset.shape

In [ ]:
# single file
file_data = np.loadtxt("data\\parsed\\NOT_SPECIFIED0.parsed", dtype=float)

print(file_data[2])